# 🔬 Screen LEAPS — Automated Discovery Engine

**What this does:** Screens the _entire US equity market_ for LEAPS call opportunities — no hardcoded ticker list.

### Pipeline

1. **Universe Screening** — yfinance equity screener → 40-50 large/mid-cap stocks across 7 sectors
2. **Fundamental Filtering** — P/E, PEG, FCF yield, margins, ROE → reject overvalued / low-quality names
3. **Technical Analysis** — Wilder's RSI, Stochastic, Bollinger Bands, MACD, MA support, Sharpe/Sortino/Calmar
4. **LEAPS Chain Scoring** — BSM P(Profit), IV vs HV edge, momentum, breakeven, style match, liquidity (6-factor model)
5. **Diversity Enforcement** — Sector caps, beta spread, correlation penalty → prevent concentration
6. **Portfolio Construction** — Round-robin Kelly-sized builder across 3 DTE buckets → final $15k book

_Style DNA: Deep-to-moderate ITM LEAPS (stock replacement), near-ATM short-term momentum plays. Preference for strong fundamentals near fair value with room to bounce/turn around within the year._


In [1]:
import time, warnings, math
from datetime import datetime, date
from typing import Optional

import numpy as np
import pandas as pd
import yfinance as yf
from yfinance import EquityQuery
from scipy.stats import norm

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
from IPython.display import display, Markdown

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 60)
pd.set_option("display.width", 200)
pd.set_option("display.max_rows", 200)
pio.renderers.default = "notebook_connected"

# ── Config ──────────────────────────────────────────────────────
STARTING_BALANCE = 15_000
RISK_FREE_RATE = 0.045  # ~4.5% annualised (T-bill proxy)
RATE_LIMIT_SLEEP = 0.35

# LEAPS parameters
LEAPS_MIN_DTE = 300
LEAPS_MAX_DTE = 540
LEAPS_TARGET_DTE = 365
MONEYNESS_RANGE = (0.80, 1.05)  # deep ITM → slightly OTM
MIN_OPEN_INTEREST = 50

# Medium-term parameters
MED_DTE_MIN = 30
MED_DTE_MAX = 90
MED_MONEYNESS = (0.90, 1.05)

# Short-term parameters
SHORT_DTE_MAX = 30

# Screener filters
SCREEN_SIZE = 50
MIN_MARKET_CAP = 2_000_000_000  # $2B+
MAX_PE = 40
MIN_ROE = 0.08
MIN_REV_GROWTH = 0.0
MIN_BETA = 0.6
MAX_BETA = 2.5
MIN_INST_HELD = 0.40
MAX_PRICE = 500

SECTORS = [
    "Communication Services",
    "Consumer Cyclical",
    "Consumer Defensive",
    "Financial Services",
    "Healthcare",
    "Industrials",
    "Technology",
]

# Diversity constraints
MAX_PER_SECTOR = 3
CORRELATION_PENALTY_THRESH = 0.65

# Portfolio construction
MAX_TICKER_PCT = 0.20
MAX_SINGLE_POS_PCT = 0.18
FRACTIONAL_KELLY = 0.25
KELLY_CAP = 0.18
EDGE_HAIRCUT = 0.90
MIN_POSITION_COST = 100

BUCKET_TARGET = {
    "⚡ <30 DTE": 0.10,
    "🔄 30-90 DTE": 0.25,
    "🏗️ 300+ DTE": 0.65,
}

# Style DNA — derived from your reference trades (GEV, KLAC, AVGO, WDC)
STYLE_PREFS = {
    "preferred_moneyness_leaps": (0.55, 1.00),
    "preferred_moneyness_swing": (0.85, 1.00),
    "ideal_moneyness_leaps": 0.78,
    "ideal_breakeven_pct": 0.045,
    "max_var_per_contract": 55_450,
    "ideal_leverage": 8.3,
}


# ── BSM helpers (used by audit cell) ──
def bsm_d1(S, K, T, r, sigma):
    return (math.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))


def bsm_theta(S, K, T, r, sigma):
    """Daily theta per share (negative for long calls)."""
    d1 = bsm_d1(S, K, T, r, sigma)
    d2 = d1 - sigma * math.sqrt(T)
    term1 = -(S * norm.pdf(d1) * sigma) / (2 * math.sqrt(T))
    term2 = -r * K * math.exp(-r * T) * norm.cdf(d2)
    return (term1 + term2) / 365


print("📋 Screen LEAPS — Automated Discovery Engine")
print(f"   Balance:  ${STARTING_BALANCE:,.0f}")
print(f"   LEAPS:   {LEAPS_MIN_DTE}–{LEAPS_MAX_DTE} DTE")
print(f"   Sectors:  {len(SECTORS)} targeted")
print(
    f"   Style:    Deep ITM, ideal moneyness ~{STYLE_PREFS['ideal_moneyness_leaps']:.0%}"
)

📋 Screen LEAPS — Automated Discovery Engine
   Balance:  $15,000
   LEAPS:   300–540 DTE
   Sectors:  7 targeted
   Style:    Deep ITM, ideal moneyness ~78%


In [ ]:
# ══════════════════════════════════════════════════════════════════
#  🔍  STEP 1: Screen the Market — Dynamic Universe Discovery
# ══════════════════════════════════════════════════════════════════

display(Markdown("# 🔍 Step 1 — Universe Screening"))
display(
    Markdown(
        "*Dynamically discovering candidates from the entire US equity market via yfinance screener.*"
    )
)

print(
    f"  Filters: MCap ≥ ${MIN_MARKET_CAP / 1e9:.0f}B | PE ≤ {MAX_PE} | ROE ≥ {MIN_ROE:.0%}"
)
print(
    f"           Beta {MIN_BETA}–{MAX_BETA} | Inst. Held ≥ {MIN_INST_HELD:.0%} | Price $10–${MAX_PRICE}"
)
print(f"           Sectors: {', '.join(SECTORS)}\n")

try:
    filters = [
        EquityQuery("eq", ["region", "us"]),
        EquityQuery("is-in", ["exchange", "NMS", "NYQ"]),
        EquityQuery("btwn", ["intradaymarketcap", MIN_MARKET_CAP, 4_000_000_000_000]),
        EquityQuery("btwn", ["intradayprice", 10, MAX_PRICE]),
        EquityQuery("btwn", ["peratio.lasttwelvemonths", 0, MAX_PE]),
        EquityQuery("gte", ["returnontotalcapital.lasttwelvemonths", MIN_ROE]),
        EquityQuery("gte", ["totalrevenues1yrgrowth.lasttwelvemonths", MIN_REV_GROWTH]),
        EquityQuery("btwn", ["beta", MIN_BETA, MAX_BETA]),
        EquityQuery("gte", ["pctheldinst", MIN_INST_HELD]),
        EquityQuery("is-in", ["sector"] + SECTORS),
    ]
    q = EquityQuery("and", filters)
    resp = yf.screen(q, size=SCREEN_SIZE, sortField="eodvolume", sortAsc=False)

    quotes = []
    if resp:
        if "quotes" in resp:
            quotes = resp["quotes"]
        elif "finance" in resp:
            result = resp.get("finance", {}).get("result", [])
            if result:
                quotes = result[0].get("quotes", [])

    UNIVERSE = list(dict.fromkeys([r["symbol"] for r in quotes if r.get("symbol")]))
    print(f"✅ Screener returned {len(UNIVERSE)} tickers")

except Exception as e:
    print(f"⚠ Screener failed ({e}). Falling back to curated large-cap universe.")
    UNIVERSE = [
        "AAPL",
        "MSFT",
        "GOOGL",
        "AMZN",
        "META",
        "NVDA",
        "AVGO",
        "CRM",
        "ORCL",
        "AMD",
        "QCOM",
        "MU",
        "TXN",
        "AMAT",
        "LRCX",
        "KLAC",
        "SNPS",
        "ADBE",
        "NOW",
        "UBER",
        "JPM",
        "GS",
        "V",
        "MA",
        "UNH",
        "JNJ",
        "LLY",
        "PFE",
        "GE",
        "CAT",
        "DE",
        "HON",
        "RTX",
        "CMI",
        "EMR",
        "WDC",
        "STX",
        "ISRG",
        "TMO",
        "ABT",
    ]

# ── Fetch spot prices, sectors, betas ──
print("\nFetching spot prices and sector data…\n")

spot_map = {}
sector_map = {}
beta_map = {}
name_map = {}
mcap_map = {}
valid_tickers = []

for ticker in UNIVERSE:
    try:
        t = yf.Ticker(ticker)
        time.sleep(RATE_LIMIT_SLEEP)
        info = t.info or {}
        hist = t.history(period="5d")
        if hist.empty:
            print(f"  ⚠ {ticker}: no price data — skipped")
            continue
        spot = float(hist["Close"].iloc[-1])
        sector = info.get("sector", "Unknown")
        beta = info.get("beta", 1.0)
        name = info.get("shortName", ticker)
        mcap = info.get("marketCap", 0)

        # Verify it has options
        try:
            exps = t.options
            if not exps or len(exps) < 3:
                print(f"  ⚠ {ticker}: insufficient expirations — skipped")
                continue
        except Exception:
            print(f"  ⚠ {ticker}: no options chain — skipped")
            continue

        spot_map[ticker] = spot
        sector_map[ticker] = sector
        beta_map[ticker] = float(beta) if beta else 1.0
        name_map[ticker] = name
        mcap_map[ticker] = mcap or 0
        valid_tickers.append(ticker)

        mcap_str = f"${mcap / 1e9:.0f}B" if mcap and mcap > 1e9 else "—"
        print(
            f"  ✅ {ticker:5s}  ${spot:>8,.2f}  β={beta_map[ticker]:.2f}  {mcap_str:>7s}  {sector}"
        )
    except Exception as e:
        print(f"  ⚠ {ticker}: {e}")

UNIVERSE = valid_tickers
print(f"\n✅ {len(UNIVERSE)} tickers with valid prices and options chains")

# ── Sector & beta breakdown ──
sector_series = pd.Series({t: sector_map[t] for t in UNIVERSE})
sector_counts = sector_series.value_counts()
beta_series = pd.Series({t: beta_map[t] for t in UNIVERSE})

display(Markdown("### 🏭 Sector & Beta Distribution"))

fig_sector = make_subplots(
    rows=1,
    cols=2,
    specs=[[{"type": "bar"}, {"type": "histogram"}]],
    subplot_titles=["By Sector", "Beta Distribution"],
)
fig_sector.add_trace(
    go.Bar(
        x=sector_counts.index,
        y=sector_counts.values,
        text=sector_counts.values,
        textposition="outside",
        marker_color=px.colors.qualitative.Set2[: len(sector_counts)],
    ),
    row=1,
    col=1,
)
fig_sector.add_trace(
    go.Histogram(x=beta_series.values, nbinsx=12, marker_color="#3498db", name="Beta"),
    row=1,
    col=2,
)
fig_sector.update_layout(
    height=350, showlegend=False, title_text=f"Universe: {len(UNIVERSE)} Tickers"
)
fig_sector.show()

# 🔍 Step 1 — Universe Screening

*Dynamically discovering candidates from the entire US equity market via yfinance screener.*

  Filters: MCap ≥ $2B | PE ≤ 40 | ROE ≥ 8%
           Beta 0.6–2.5 | Inst. Held ≥ 40% | Price $10–$500
           Sectors: Communication Services, Consumer Cyclical, Consumer Defensive, Financial Services, Healthcare, Industrials, Technology

✅ Screener returned 50 tickers

Fetching spot prices and sector data…

  ✅ AMZN   $  210.32  β=1.39   $2258B  Consumer Cyclical
  ✅ GOOGL  $  322.86  β=1.09   $3906B  Communication Services
  ✅ AAL    $   15.24  β=1.23     $10B  Industrials
  ✅ F      $   13.80  β=1.67     $55B  Consumer Cyclical
  ✅ MSFT   $  401.14  β=1.08   $2981B  Technology
  ✅ SMCI   $   34.38  β=1.52     $21B  Technology
  ✅ NFLX   $   82.20  β=1.71    $349B  Communication Services
  ✅ T      $   27.13  β=0.61    $192B  Communication Services


Exception ignored from cffi callback <function buffer_callback at 0x1124fd640>:
Traceback (most recent call last):
  File "/Users/matthewwoods/Development/options-screener/.venv/lib/python3.14/site-packages/curl_cffi/curl.py", line 100, in buffer_callback
    @ffi.def_extern()
KeyboardInterrupt: 


  ⚠ MU: no options chain — skipped
  ✅ PYPL   $   40.42  β=1.42     $38B  Financial Services
  ✅ PATH   $   12.89  β=1.07      $7B  Technology
  ✅ GOOG   $  323.10  β=1.09   $3909B  Communication Services
  ✅ ORCL   $  142.82  β=1.63    $410B  Technology
  ✅ CCL    $   33.99  β=2.44     $47B  Consumer Cyclical
  ✅ WMT    $  131.18  β=0.67   $1046B  Consumer Defensive
  ✅ UBER   $   74.77  β=1.21    $156B  Technology
  ✅ CSCO   $   84.82  β=0.86    $335B  Technology
  ✅ DOCS   $   27.73  β=1.39      $5B  Healthcare
  ✅ MRVL   $   80.28  β=1.98     $69B  Technology
  ✅ NCLH   $   23.32  β=2.03     $11B  Consumer Cyclical
  ✅ ASX    $   20.89  β=1.07     $46B  Technology
  ✅ CMG    $   39.39  β=1.00     $52B  Consumer Cyclical
  ✅ TSM    $  348.85  β=1.27   $1809B  Technology


: 

---

## 📊 Step 2 — Fundamentals & Valuation Gate

Fetch forward P/E, PEG, FCF yield, revenue growth, profit margins, ROE for every ticker.
Score each 0–100 on valuation attractiveness. **Reject** tickers that fail ≥ 2/3 of value checks (overvalued traps).
Only companies near or under fair value with quality fundamentals survive.


In [ ]:
# ══════════════════════════════════════════════════════════════════
#  📊  STEP 2: Fundamentals & Valuation Gate
# ══════════════════════════════════════════════════════════════════

print("Fetching fundamentals (P/E, PEG, FCF, margins, ROE)…\n")

fundamentals = {}
for ticker in UNIVERSE:
    try:
        t = yf.Ticker(ticker)
        time.sleep(RATE_LIMIT_SLEEP)
        info = t.info or {}

        fwd_pe = info.get("forwardPE") or info.get("forwardPe")
        trail_pe = info.get("trailingPE") or info.get("trailingPe")
        peg = info.get("pegRatio") or info.get("trailingPegRatio")
        mcap = info.get("marketCap", 0)
        fcf = info.get("freeCashflow", 0)
        ev = info.get("enterpriseValue", 0)
        rev_growth = info.get("revenueGrowth")
        profit_margin = info.get("profitMargins")
        gross_margin = info.get("grossMargins")
        op_margin = info.get("operatingMargins")
        roe = info.get("returnOnEquity")
        debt_to_equity = info.get("debtToEquity")
        total_debt = info.get("totalDebt", 0)
        total_cash = info.get("totalCash", 0)
        ebitda = info.get("ebitda", 0)
        ps = info.get("priceToSalesTrailing12Months")

        fcf_yield = (fcf / mcap * 100) if (mcap and fcf) else None
        net_debt = (total_debt - total_cash) if (total_debt and total_cash) else None
        nd_ebitda = (
            (net_debt / ebitda)
            if (net_debt is not None and ebitda and ebitda > 0)
            else None
        )

        fundamentals[ticker] = {
            "fwd_pe": fwd_pe,
            "trail_pe": trail_pe,
            "peg": peg,
            "mcap": mcap,
            "fcf_yield": fcf_yield,
            "rev_growth": rev_growth,
            "profit_margin": profit_margin,
            "gross_margin": gross_margin,
            "op_margin": op_margin,
            "roe": roe,
            "debt_to_equity": debt_to_equity,
            "ps": ps,
            "nd_ebitda": nd_ebitda,
        }

        pe_s = f"FwdPE={fwd_pe:.1f}" if fwd_pe else "FwdPE=—"
        peg_s = f"PEG={peg:.2f}" if peg else "PEG=—"
        fcf_s = f"FCFy={fcf_yield:.1f}%" if fcf_yield else "FCFy=—"
        print(
            f"  {ticker:5s}  {pe_s:14s}  {peg_s:10s}  {fcf_s:10s}  Margin={profit_margin:.0%}"
            if profit_margin
            else f"  {ticker:5s}  {pe_s:14s}  {peg_s:10s}  {fcf_s:10s}"
        )
    except Exception as e:
        print(f"  ⚠ {ticker}: {e}")

print(f"\n✅ Fundamentals for {len(fundamentals)} tickers")


# ──────────────────────────────────────────────────────────────────
# Valuation Score (0–100) — higher = more attractively valued
# ──────────────────────────────────────────────────────────────────


def valuation_score(ticker):
    f = fundamentals.get(ticker, {})
    if not f:
        return 50
    score = 0
    n = 0

    fwd_pe = f.get("fwd_pe")
    if fwd_pe and fwd_pe > 0:
        if fwd_pe < 12:
            score += 85
        elif fwd_pe < 18:
            score += 75
        elif fwd_pe < 25:
            score += 60
        elif fwd_pe < 35:
            score += 40
        elif fwd_pe < 50:
            score += 25
        else:
            score += 10
        n += 1

    peg = f.get("peg")
    if peg and peg > 0:
        if peg < 0.8:
            score += 90
        elif peg < 1.2:
            score += 75
        elif peg < 2.0:
            score += 50
        elif peg < 3.0:
            score += 30
        else:
            score += 10
        n += 1

    fcf_y = f.get("fcf_yield")
    if fcf_y is not None:
        if fcf_y > 6:
            score += 90
        elif fcf_y > 4:
            score += 75
        elif fcf_y > 2:
            score += 55
        elif fcf_y > 0:
            score += 35
        else:
            score += 10
        n += 1

    rg = f.get("rev_growth")
    if rg is not None:
        if rg > 0.20:
            score += 85
        elif rg > 0.10:
            score += 65
        elif rg > 0.05:
            score += 50
        elif rg > 0:
            score += 35
        else:
            score += 15
        n += 1

    pm = f.get("profit_margin")
    if pm is not None:
        if pm > 0.30:
            score += 85
        elif pm > 0.20:
            score += 70
        elif pm > 0.10:
            score += 50
        elif pm > 0:
            score += 30
        else:
            score += 10
        n += 1

    return score / n if n > 0 else 50


# ──────────────────────────────────────────────────────────────────
# Value / Quality Gate — reject poor fundamentals
# ──────────────────────────────────────────────────────────────────


def value_quality_gate(ticker):
    """Returns (pass: bool, value_ratio, quality_ratio, reasons)."""
    f = fundamentals.get(ticker, {})
    if not f:
        return False, 0, 0, ["No data"]

    # Value checks
    val_checks = {
        "pe": f.get("fwd_pe") is not None and 0 < f["fwd_pe"] <= 35,
        "ps": f.get("ps") is not None and f["ps"] <= 8,
        "fcf": f.get("fcf_yield") is not None and f["fcf_yield"] >= 1.5,
        "nd_ebitda": f.get("nd_ebitda") is None or f["nd_ebitda"] <= 3.0,
    }
    val_avail = sum(
        1
        for k in ["pe", "ps", "fcf", "nd_ebitda"]
        if f.get(
            k.replace("pe", "fwd_pe")
            .replace("ps", "ps")
            .replace("fcf", "fcf_yield")
            .replace("nd_ebitda", "nd_ebitda")
        )
        is not None
    )
    val_avail = max(
        val_avail,
        sum(
            1
            for v in [
                f.get("fwd_pe"),
                f.get("ps"),
                f.get("fcf_yield"),
                f.get("nd_ebitda"),
            ]
            if v is not None
        ),
    )
    val_pass = sum(v for v in val_checks.values())
    val_ratio = val_pass / val_avail if val_avail > 0 else 0

    # Quality checks
    qual_checks = {
        "roe": f.get("roe") is not None and f["roe"] >= 0.08,
        "margin": f.get("profit_margin") is not None and f["profit_margin"] >= 0.08,
        "gross": f.get("gross_margin") is not None and f["gross_margin"] >= 0.25,
        "growth": f.get("rev_growth") is not None and f["rev_growth"] >= 0.0,
    }
    qual_avail = sum(
        1
        for v in [
            f.get("roe"),
            f.get("profit_margin"),
            f.get("gross_margin"),
            f.get("rev_growth"),
        ]
        if v is not None
    )
    qual_pass = sum(v for v in qual_checks.values())
    qual_ratio = qual_pass / qual_avail if qual_avail > 0 else 0

    reasons = []
    if val_ratio < 0.50:
        reasons.append(f"Value fail ({val_pass}/{val_avail})")
    if qual_ratio < 0.50:
        reasons.append(f"Quality fail ({qual_pass}/{qual_avail})")

    passed = val_ratio >= 0.50 and qual_ratio >= 0.50
    return passed, val_ratio, qual_ratio, reasons


# ── Apply gate ──
val_scores = {}
gate_results = {}
rejected = []
survivors = []

for ticker in UNIVERSE:
    vs = valuation_score(ticker)
    val_scores[ticker] = vs
    passed, vr, qr, reasons = value_quality_gate(ticker)
    gate_results[ticker] = {
        "passed": passed,
        "val_ratio": vr,
        "qual_ratio": qr,
        "reasons": reasons,
    }

    if passed:
        survivors.append(ticker)
        print(f"  ✅ {ticker:5s}  ValScore={vs:.0f}  Value={vr:.0%}  Quality={qr:.0%}")
    else:
        rejected.append(ticker)
        print(f"  ❌ {ticker:5s}  ValScore={vs:.0f}  {' | '.join(reasons)}")

print(f"\n✅ {len(survivors)} passed / {len(rejected)} rejected")
UNIVERSE = survivors

# ── Valuation Leaderboard ──
display(Markdown("### 📊 Valuation Leaderboard"))

val_df = pd.DataFrame(
    [
        {
            "Ticker": t,
            "Sector": sector_map.get(t, "—"),
            "β": beta_map.get(t, 1.0),
            "Val Score": val_scores[t],
            "Fwd PE": fundamentals.get(t, {}).get("fwd_pe"),
            "PEG": fundamentals.get(t, {}).get("peg"),
            "FCF Yield": fundamentals.get(t, {}).get("fcf_yield"),
            "Rev Growth": fundamentals.get(t, {}).get("rev_growth"),
            "Profit Margin": fundamentals.get(t, {}).get("profit_margin"),
            "ROE": fundamentals.get(t, {}).get("roe"),
            "Val Gate": gate_results[t]["val_ratio"],
            "Qual Gate": gate_results[t]["qual_ratio"],
        }
        for t in UNIVERSE
    ]
).sort_values("Val Score", ascending=False)

display(
    val_df.style.format(
        {
            "Val Score": "{:.0f}",
            "β": "{:.2f}",
            "Fwd PE": "{:.1f}",
            "PEG": "{:.2f}",
            "FCF Yield": "{:.1f}%",
            "Rev Growth": "{:.1%}",
            "Profit Margin": "{:.1%}",
            "ROE": "{:.1%}",
            "Val Gate": "{:.0%}",
            "Qual Gate": "{:.0%}",
        },
        na_rep="—",
    )
    .background_gradient(subset=["Val Score"], cmap="RdYlGn", vmin=30, vmax=80)
    .background_gradient(
        subset=["Val Gate", "Qual Gate"], cmap="RdYlGn", vmin=0.3, vmax=1.0
    )
    .set_caption("Survivors — passed value + quality gate")
)

# ── Valuation Components Chart ──
if len(val_df) > 0:
    plot_df = val_df.head(20).copy()
    fig_val = go.Figure()
    for col, color in [
        ("Fwd PE", "#e74c3c"),
        ("PEG", "#3498db"),
        ("FCF Yield", "#2ecc71"),
    ]:
        if col in plot_df.columns:
            vals = plot_df[col].fillna(0)
            if col == "FCF Yield":
                vals = vals  # already in %
            fig_val.add_trace(
                go.Bar(name=col, x=plot_df["Ticker"], y=vals, marker_color=color)
            )
    fig_val.update_layout(
        barmode="group",
        title="Valuation Components — Top 20",
        height=400,
        yaxis_title="Value",
    )
    fig_val.show()

---

## 📈 Step 3 — Technical Analysis & Bounce Scanner

For every surviving ticker: HV (30/60-day), Wilder's RSI, moving averages, golden cross detection.
Full bounce scanner: RSI, Stochastic %K/%D, Bollinger Bands %B, MACD histogram, MA support, 52W high distance, volume capitulation.
Plus Sharpe / Sortino / Calmar / Win Rate from 6-month daily returns.

**Goal:** Identify names with technical room to bounce/recover and strong risk-adjusted momentum.


In [ ]:
# ══════════════════════════════════════════════════════════════════
#  📈  STEP 3: HV / Momentum / Bounce Scanner / Ratios
# ══════════════════════════════════════════════════════════════════

# ── 3A: HV & Momentum ──────────────────────────────────────────
print("Computing HV / Momentum / Trend…\n")

hv_momentum = {}

for ticker in UNIVERSE:
    try:
        t = yf.Ticker(ticker)
        time.sleep(RATE_LIMIT_SLEEP)
        hist = t.history(period="1y")
        if hist.empty or "Close" not in hist.columns:
            continue
        closes = hist["Close"].dropna()
        if len(closes) < 60:
            continue

        log_ret = np.log(closes / closes.shift(1)).dropna()
        hv_30 = float(log_ret.iloc[-30:].std() * math.sqrt(252))
        hv_60 = float(log_ret.iloc[-60:].std() * math.sqrt(252))

        price = float(closes.iloc[-1])
        ret_1m = (
            float(closes.iloc[-1] / closes.iloc[-21] - 1) if len(closes) > 21 else 0
        )
        ret_3m = (
            float(closes.iloc[-1] / closes.iloc[-63] - 1) if len(closes) > 63 else 0
        )
        ret_6m = (
            float(closes.iloc[-1] / closes.iloc[-126] - 1) if len(closes) > 126 else 0
        )

        # RSI (Wilder's EWM com=13)
        delta = closes.diff()
        gain = delta.clip(lower=0).ewm(com=13, adjust=False).mean()
        loss_s = (-delta.clip(upper=0)).ewm(com=13, adjust=False).mean()
        rs = gain.iloc[-1] / loss_s.iloc[-1] if loss_s.iloc[-1] > 0 else 100
        rsi = float(100 - (100 / (1 + rs)))

        ma50 = float(closes.rolling(50).mean().iloc[-1]) if len(closes) >= 50 else None
        ma200 = (
            float(closes.rolling(200).mean().iloc[-1]) if len(closes) >= 200 else None
        )
        above_50 = price > ma50 if ma50 else False
        above_200 = price > ma200 if ma200 else False
        golden_cross = (ma50 > ma200) if (ma50 and ma200) else False

        high_52w = (
            float(closes.iloc[-252:].max())
            if len(closes) >= 252
            else float(closes.max())
        )
        dist_52w = (price - high_52w) / high_52w

        hv_momentum[ticker] = {
            "hv_30": hv_30,
            "hv_60": hv_60,
            "ret_1m": ret_1m,
            "ret_3m": ret_3m,
            "ret_6m": ret_6m,
            "rsi": rsi,
            "dist_52w_high": dist_52w,
            "above_50ma": above_50,
            "above_200ma": above_200,
            "golden_cross": golden_cross,
        }
        flags = []
        if above_50:
            flags.append("▲50MA")
        if above_200:
            flags.append("▲200MA")
        if golden_cross:
            flags.append("🔥GC")
        print(
            f"  {ticker:5s}: HV30={hv_30:.1%} 1M={ret_1m:+.1%} 3M={ret_3m:+.1%} RSI={rsi:.0f} {' '.join(flags)}"
        )
    except Exception as e:
        print(f"  ⚠ {ticker}: {e}")

print(f"\n✅ HV/momentum for {len(hv_momentum)} tickers")


# ── 3B: Full Bounce Scanner ────────────────────────────────────
print("\n🔎 Bounce Scanner — technical reversal signals…\n")

bounce_rows = []
for ticker in UNIVERSE:
    try:
        t = yf.Ticker(ticker)
        time.sleep(RATE_LIMIT_SLEEP)
        hist = t.history(period="1y")
        if hist.empty or len(hist) < 60:
            continue

        closes = hist["Close"].dropna()
        highs = hist["High"].dropna()
        lows = hist["Low"].dropna()
        volumes = hist["Volume"].dropna()
        price = float(closes.iloc[-1])
        spot = spot_map.get(ticker, price)

        # RSI (Wilder's EWM)
        d = closes.diff()
        g = d.clip(lower=0).ewm(com=13, adjust=False).mean()
        ls = (-d.clip(upper=0)).ewm(com=13, adjust=False).mean()
        rs_v = g.iloc[-1] / ls.iloc[-1] if ls.iloc[-1] > 0 else 100
        rsi = float(100 - (100 / (1 + rs_v)))

        # Stochastic %K/%D
        low_14 = lows.rolling(14).min()
        high_14 = highs.rolling(14).max()
        stoch_k = (
            float(
                ((price - low_14.iloc[-1]) / (high_14.iloc[-1] - low_14.iloc[-1])) * 100
            )
            if (high_14.iloc[-1] - low_14.iloc[-1]) > 0
            else 50
        )
        stoch_k_s = ((closes - low_14) / (high_14 - low_14)) * 100
        stoch_d = float(stoch_k_s.rolling(3).mean().iloc[-1])
        stoch_cross_up = stoch_k > stoch_d and stoch_k < 30

        # Bollinger Bands
        bb_mid = closes.rolling(20).mean()
        bb_std = closes.rolling(20).std()
        bb_upper = bb_mid + 2 * bb_std
        bb_lower = bb_mid - 2 * bb_std
        bb_pct_b = (
            float((price - bb_lower.iloc[-1]) / (bb_upper.iloc[-1] - bb_lower.iloc[-1]))
            if (bb_upper.iloc[-1] - bb_lower.iloc[-1]) > 0
            else 0.5
        )
        below_lower_bb = price < bb_lower.iloc[-1]

        # MACD
        ema12 = closes.ewm(span=12).mean()
        ema26 = closes.ewm(span=26).mean()
        macd_line = ema12 - ema26
        macd_signal = macd_line.ewm(span=9).mean()
        macd_hist = macd_line - macd_signal
        macd_hist_val = float(macd_hist.iloc[-1])
        macd_hist_prev = (
            float(macd_hist.iloc[-2]) if len(macd_hist) > 1 else macd_hist_val
        )
        macd_turning_up = macd_hist_val > macd_hist_prev and macd_hist_val < 0

        # MAs
        ma20 = float(bb_mid.iloc[-1]) if not pd.isna(bb_mid.iloc[-1]) else None
        ma50 = float(closes.rolling(50).mean().iloc[-1]) if len(closes) >= 50 else None
        ma200 = (
            float(closes.rolling(200).mean().iloc[-1]) if len(closes) >= 200 else None
        )
        dist_ma200 = (price - ma200) / ma200 if ma200 else np.nan
        dist_ma50 = (price - ma50) / ma50 if ma50 else np.nan

        # 52W
        high_52w = float(closes.max())
        low_52w = float(closes.min())
        dist_high = (price - high_52w) / high_52w

        # Volume
        avg_vol_20 = float(volumes.rolling(20).mean().iloc[-1])
        recent_vol = float(volumes.iloc[-5:].mean())
        vol_ratio = recent_vol / avg_vol_20 if avg_vol_20 > 0 else 1.0

        # Returns
        ret_1w = float(price / closes.iloc[-5] - 1) if len(closes) > 5 else 0
        ret_1m = float(price / closes.iloc[-21] - 1) if len(closes) > 21 else 0

        # ── BOUNCE SCORE ──
        score = 0
        signals = []

        if rsi < 20:
            score += 25
            signals.append(f"RSI extreme ({rsi:.0f})")
        elif rsi < 30:
            score += 20
            signals.append(f"RSI oversold ({rsi:.0f})")
        elif rsi < 40:
            score += 10
            signals.append(f"RSI weak ({rsi:.0f})")

        if stoch_cross_up:
            score += 15
            signals.append("Stoch cross up from oversold")
        elif stoch_k < 20:
            score += 10
            signals.append(f"Stoch oversold ({stoch_k:.0f})")

        if below_lower_bb:
            score += 15
            signals.append("Below lower BB")
        elif bb_pct_b < 0.15:
            score += 10
            signals.append(f"Near lower BB (%B={bb_pct_b:.2f})")

        if macd_turning_up:
            score += 12
            signals.append("MACD histogram turning up")
        elif macd_hist_val > 0 and macd_hist_prev < 0:
            score += 10
            signals.append("MACD crossed positive")

        if ma200 and abs(dist_ma200) < 0.03:
            score += 12
            signals.append(f"At 200MA support ({dist_ma200:+.1%})")
        elif ma50 and abs(dist_ma50) < 0.03:
            score += 8
            signals.append(f"At 50MA ({dist_ma50:+.1%})")
        elif ma200 and dist_ma200 > 0:
            score += 5
            signals.append("Above 200MA")

        if -0.25 < dist_high < -0.08:
            score += 10
            signals.append(f"Healthy pullback ({dist_high:.0%} off 52W high)")
        elif -0.35 < dist_high < -0.25:
            score += 6
            signals.append(f"Deep pullback ({dist_high:.0%})")

        if vol_ratio > 1.5 and ret_1w < -0.03:
            score += 8
            signals.append(f"Volume spike ({vol_ratio:.1f}x)")
        elif vol_ratio > 1.2:
            score += 4
            signals.append(f"Elevated vol ({vol_ratio:.1f}x)")

        if ret_1w < -0.08:
            score -= 10
            signals.append("⚠ Still falling")

        hv = hv_momentum.get(ticker, {})
        if hv.get("golden_cross") and rsi < 40:
            score += 5
            signals.append("Golden cross on pullback")

        score = max(0, min(100, score))

        bounce_rows.append(
            {
                "Ticker": ticker,
                "Sector": sector_map.get(ticker, "—"),
                "Spot": spot,
                "RSI": rsi,
                "Stoch %K": stoch_k,
                "BB %B": bb_pct_b,
                "MACD Hist": macd_hist_val,
                "MACD Turning": "✅" if macd_turning_up else "—",
                "Dist 200MA": dist_ma200,
                "Dist 52W High": dist_high,
                "Vol Ratio": vol_ratio,
                "Bounce Score": score,
                "Signals": " | ".join(signals) if signals else "No strong signals",
            }
        )
        status = "🟢" if score >= 50 else "🟡" if score >= 30 else "⚪"
        print(
            f"  {status} {ticker:5s}: Bounce={score} RSI={rsi:.0f} Stoch={stoch_k:.0f} BB%B={bb_pct_b:.2f} 1M={ret_1m:+.1%}"
        )

    except Exception as e:
        print(f"  ⚠ {ticker}: {e}")

bounce_df = pd.DataFrame(bounce_rows).sort_values("Bounce Score", ascending=False)
print(f"\n✅ Scanned {len(bounce_df)} tickers for bounce setups")


# ── 3C: Sharpe / Sortino / Calmar / Win Rate ──────────────────
print("\nComputing Sharpe / Sortino / Calmar / Win Rate …\n")

ticker_ratios = {}
for ticker in UNIVERSE:
    try:
        t = yf.Ticker(ticker)
        time.sleep(RATE_LIMIT_SLEEP)
        hist = t.history(period="6mo")
        if hist.empty or len(hist) < 60:
            continue
        closes = hist["Close"].dropna()
        daily_ret = closes.pct_change().dropna()

        ann_ret = float(daily_ret.mean() * 252)
        ann_vol = float(daily_ret.std() * np.sqrt(252))
        sharpe = (ann_ret - RISK_FREE_RATE) / ann_vol if ann_vol > 0 else 0

        daily_rf = RISK_FREE_RATE / 252
        diff_below = np.minimum(daily_ret - daily_rf, 0)
        down_vol = (
            float(np.sqrt(np.mean(diff_below**2)) * np.sqrt(252))
            if len(daily_ret) > 20
            else ann_vol
        )
        sortino = (ann_ret - RISK_FREE_RATE) / down_vol if down_vol > 0 else 0

        cum = (1 + daily_ret).cumprod()
        peak = cum.cummax()
        drawdown = (cum - peak) / peak
        max_dd = float(drawdown.min())
        calmar = ann_ret / abs(max_dd) if max_dd != 0 else 0

        win_rate = float((daily_ret > 0).sum() / len(daily_ret))
        beta_approx = ann_vol / 0.16

        ticker_ratios[ticker] = {
            "ann_ret": ann_ret,
            "ann_vol": ann_vol,
            "sharpe": sharpe,
            "sortino": sortino,
            "calmar": calmar,
            "max_dd": max_dd,
            "win_rate": win_rate,
            "beta": beta_approx,
        }
        flag = "🟢" if sharpe > 0.5 else "🟡" if sharpe > 0 else "🔴"
        print(
            f"  {flag} {ticker:5s}  Sharpe={sharpe:+.2f}  Sortino={sortino:+.2f}  Calmar={calmar:+.2f}  MaxDD={max_dd:+.1%}  WinRate={win_rate:.0%}"
        )
    except Exception as e:
        print(f"  ⚠ {ticker}: {e}")

print(f"\n✅ Ratios computed for {len(ticker_ratios)} tickers")


# ── Display: Combined Technical Dashboard ──
display(Markdown("### 📈 Technical Dashboard"))

tech_rows = []
for ticker in UNIVERSE:
    hv = hv_momentum.get(ticker, {})
    tr = ticker_ratios.get(ticker, {})
    br = (
        bounce_df[bounce_df["Ticker"] == ticker].iloc[0]
        if ticker in bounce_df["Ticker"].values
        else {}
    )

    tech_rows.append(
        {
            "Ticker": ticker,
            "Sector": sector_map.get(ticker, "—"),
            "HV30": hv.get("hv_30"),
            "RSI": hv.get("rsi"),
            "1M Ret": hv.get("ret_1m"),
            "3M Ret": hv.get("ret_3m"),
            "Sharpe": tr.get("sharpe"),
            "Sortino": tr.get("sortino"),
            "Max DD": tr.get("max_dd"),
            "Win Rate": tr.get("win_rate"),
            "Bounce": br.get("Bounce Score", 0)
            if isinstance(br, dict)
            else (
                br["Bounce Score"]
                if hasattr(br, "__getitem__") and "Bounce Score" in br
                else 0
            ),
            "Val Score": val_scores.get(ticker, 50),
        }
    )

tech_df = pd.DataFrame(tech_rows).sort_values("Bounce", ascending=False)

display(
    tech_df.style.format(
        {
            "HV30": "{:.1%}",
            "RSI": "{:.0f}",
            "1M Ret": "{:+.1%}",
            "3M Ret": "{:+.1%}",
            "Sharpe": "{:+.2f}",
            "Sortino": "{:+.2f}",
            "Max DD": "{:.0%}",
            "Win Rate": "{:.0%}",
            "Bounce": "{:.0f}",
            "Val Score": "{:.0f}",
        },
        na_rep="—",
    )
    .background_gradient(subset=["Bounce"], cmap="RdYlGn", vmin=0, vmax=60)
    .background_gradient(subset=["Sharpe"], cmap="RdYlGn", vmin=-1, vmax=2)
    .background_gradient(subset=["Val Score"], cmap="RdYlGn", vmin=30, vmax=80)
    .set_caption("Technical + Valuation — combined view")
)

# ── Radar Charts for Top 6 ──
top6 = tech_df.head(6)["Ticker"].tolist()
if top6:
    categories = ["Val Score", "Bounce", "Sharpe×30", "Sortino×20", "Win%×100"]
    fig_radar = go.Figure()
    for ticker in top6:
        r = tech_df[tech_df["Ticker"] == ticker].iloc[0]
        vals = [
            r.get("Val Score", 50),
            r.get("Bounce", 0),
            max(0, (r.get("Sharpe", 0) + 1) * 30),
            max(0, (r.get("Sortino", 0) + 1) * 20),
            (r.get("Win Rate", 0.5) or 0.5) * 100,
        ]
        fig_radar.add_trace(
            go.Scatterpolar(r=vals, theta=categories, fill="toself", name=ticker)
        )
    fig_radar.update_layout(
        polar=dict(radialaxis=dict(visible=True, range=[0, 100])),
        title="Top 6 Tickers — Radar Comparison",
        height=500,
    )
    fig_radar.show()

---

## 🎯 Step 4 — LEAPS Chain Fetch & Edge Scoring

Fetch 300–540 DTE call chains for every survivor. Score every option with the **6-factor model**:

1. **P(Profit)** — BSM risk-neutral probability of breakeven (25%)
2. **Momentum** — 1M/3M returns, RSI sweet spot, MA trend (20%)
3. **Breakeven %** — how far stock must move to profit (15%)
4. **IV vs HV Edge** — is IV cheap relative to realized? (15%)
5. **Style Match** — moneyness vs your DNA preference (15%)
6. **Liquidity** — open interest, bid-ask spread (10%)

Also fetches medium-term (30–90 DTE) and short-term (<30 DTE) chains for the unified builder.


In [ ]:
# ══════════════════════════════════════════════════════════════════
#  🎯  STEP 4: Fetch LEAPS + Medium + Short Chains & Score
# ══════════════════════════════════════════════════════════════════


def safe_float(v, default=0):
    try:
        return (
            float(v)
            if v is not None and not (isinstance(v, float) and np.isnan(v))
            else default
        )
    except (TypeError, ValueError):
        return default


def bsm_prob_profit(spot, breakeven, iv, dte, r=None):
    """P(spot > breakeven at expiry) under BSM lognormal model."""
    if r is None:
        r = RISK_FREE_RATE
    if iv <= 0 or dte <= 0 or spot <= 0 or breakeven <= 0:
        return np.nan
    T = dte / 365.0
    d2 = (math.log(spot / breakeven) + (r - 0.5 * iv**2) * T) / (iv * math.sqrt(T))
    return float(norm.cdf(d2))


def score_option(row, hv_data, style_prefs):
    """Score a single option 0-100 based on 6 edge factors."""
    scores = {}
    weights = {}
    ticker = row["ticker"]
    hv = hv_data.get(ticker, {})
    if not hv:
        return np.nan, np.nan, {}

    spot = row["spot"]
    iv = row.get("iv", np.nan)
    dte = row["dte"]
    breakeven = row["breakeven"]
    moneyness = row["moneyness"]

    # 1. P(Profit) — 25%
    p_profit = bsm_prob_profit(spot, breakeven, iv, dte) if not np.isnan(iv) else np.nan
    if not np.isnan(p_profit):
        scores["p_profit"] = min(p_profit * 100, 100)
        weights["p_profit"] = 0.25

    # 2. Momentum — 20%
    ret_1m = hv.get("ret_1m", 0)
    ret_3m = hv.get("ret_3m", 0)
    rsi = hv.get("rsi", 50)
    above_50 = hv.get("above_50ma", False)
    above_200 = hv.get("above_200ma", False)
    golden_cross = hv.get("golden_cross", False)

    mom_score = 0
    mom_score += np.clip(ret_1m * 200, -30, 40)
    mom_score += np.clip(ret_3m * 100, -20, 30)
    if 45 < rsi < 72:
        mom_score += 15
    elif rsi >= 72:
        mom_score += 5
    if above_50:
        mom_score += 5
    if above_200:
        mom_score += 5
    if golden_cross:
        mom_score += 5
    scores["momentum"] = np.clip(mom_score, 0, 100)
    weights["momentum"] = 0.20

    # 3. Breakeven % — 15%
    be_pct = row.get("breakeven_pct", np.nan)
    if not np.isnan(be_pct):
        be_score = np.clip((0.15 - be_pct) / 0.20 * 100, 0, 100)
        scores["breakeven"] = be_score
        weights["breakeven"] = 0.15

    # 4. IV vs HV edge — 15%
    hv_30 = hv.get("hv_30", np.nan)
    if not np.isnan(iv) and not np.isnan(hv_30) and iv > 0:
        iv_hv_ratio = iv / hv_30 if hv_30 > 0 else 2.0
        if iv_hv_ratio < 0.9:
            iv_score = 90
        elif iv_hv_ratio < 1.1:
            iv_score = 70
        elif iv_hv_ratio < 1.3:
            iv_score = 45
        else:
            iv_score = max(0, 30 - (iv_hv_ratio - 1.3) * 50)
        scores["iv_edge"] = iv_score
        weights["iv_edge"] = 0.15

    # 5. Style match — 15%
    ideal_mon = style_prefs.get("ideal_moneyness_leaps", 0.78)
    mon_dist = abs(moneyness - ideal_mon)
    style_score = max(0, 100 - mon_dist * 500)
    if moneyness < 0.90:
        style_score = min(100, style_score + 10)
    scores["style_match"] = style_score
    weights["style_match"] = 0.15

    # 6. Liquidity — 10%
    oi = safe_float(row.get("openInterest", 0))
    spread_pct = row.get("spread_pct", np.nan)
    liq_score = 0
    if oi > 500:
        liq_score += 40
    elif oi > 100:
        liq_score += 25
    elif oi > 20:
        liq_score += 10
    if not np.isnan(spread_pct):
        if spread_pct < 0.05:
            liq_score += 60
        elif spread_pct < 0.10:
            liq_score += 40
        elif spread_pct < 0.20:
            liq_score += 20
    scores["liquidity"] = min(liq_score, 100)
    weights["liquidity"] = 0.10

    total_w = sum(weights.values())
    if total_w == 0:
        return np.nan, p_profit, scores
    edge_score = sum(scores[k] * weights[k] for k in scores) / total_w
    return edge_score, p_profit, scores


# ── Fetch LEAPS chains ──
print("Fetching LEAPS call chains (300–540 DTE)…\n")

today = date.today()
all_chains = []
skipped = []

for ticker in UNIVERSE:
    try:
        t = yf.Ticker(ticker)
        time.sleep(RATE_LIMIT_SLEEP)
        all_exps = t.options
        if not all_exps:
            skipped.append((ticker, "no exps"))
            continue

        spot = spot_map[ticker]

        # Categorise expirations
        leaps_exps, med_exps, short_exps = [], [], []
        for exp_str in all_exps:
            try:
                exp_date = datetime.strptime(exp_str, "%Y-%m-%d").date()
                dte = (exp_date - today).days
                if dte <= 0:
                    continue
                if LEAPS_MIN_DTE <= dte <= LEAPS_MAX_DTE:
                    leaps_exps.append((exp_str, dte))
                elif MED_DTE_MIN <= dte <= MED_DTE_MAX:
                    med_exps.append((exp_str, dte))
                elif 5 <= dte < SHORT_DTE_MAX:
                    short_exps.append((exp_str, dte))
            except ValueError:
                continue

        fetched = 0
        for bucket_type, exp_list, mon_range in [
            ("LEAPS", leaps_exps, MONEYNESS_RANGE),
            ("MED", med_exps, MED_MONEYNESS),
            ("SHORT", short_exps, (0.95, 1.05)),
        ]:
            for exp_str, dte in exp_list[:2]:  # max 2 exps per bucket per ticker
                try:
                    chain = t.option_chain(exp_str)
                    time.sleep(RATE_LIMIT_SLEEP)
                    calls = chain.calls
                    if calls.empty:
                        continue

                    for _, opt in calls.iterrows():
                        strike = safe_float(opt.get("strike"))
                        if strike <= 0:
                            continue
                        moneyness = strike / spot
                        if not (mon_range[0] <= moneyness <= mon_range[1]):
                            continue

                        bid = safe_float(opt.get("bid"), 0)
                        ask = safe_float(opt.get("ask"), 0)
                        mid = (
                            (bid + ask) / 2
                            if bid > 0 and ask > 0
                            else safe_float(opt.get("lastPrice"), 0)
                        )
                        if mid <= 0:
                            continue
                        iv = safe_float(opt.get("impliedVolatility"), np.nan)
                        oi_val = safe_float(opt.get("openInterest"), 0)
                        spread_pct = (
                            (ask - bid) / mid
                            if mid > 0 and bid > 0 and ask > 0
                            else np.nan
                        )

                        breakeven = strike + mid
                        breakeven_pct = (breakeven - spot) / spot
                        leverage = spot / mid if mid > 0 else 0
                        cost_per = mid * 100

                        all_chains.append(
                            {
                                "ticker": ticker,
                                "expiration": exp_str,
                                "dte": dte,
                                "strike": strike,
                                "spot": spot,
                                "moneyness": moneyness,
                                "bid": bid,
                                "ask": ask,
                                "mid": mid,
                                "iv": iv if iv > 0 else np.nan,
                                "openInterest": oi_val,
                                "spread_pct": spread_pct,
                                "breakeven": breakeven,
                                "breakeven_pct": breakeven_pct,
                                "leverage": leverage,
                                "cost_per_contract": cost_per,
                                "bucket_type": bucket_type,
                            }
                        )
                        fetched += 1
                except Exception:
                    continue

        print(f"  {ticker:5s}: {fetched} options across LEAPS/MED/SHORT")
    except Exception as e:
        print(f"  ⚠ {ticker}: {e}")
        skipped.append((ticker, str(e)))

chain_df = pd.DataFrame(all_chains)
print(f"\n✅ {len(chain_df)} total options from {chain_df['ticker'].nunique()} tickers")
print(f"   LEAPS: {len(chain_df[chain_df['bucket_type'] == 'LEAPS'])}")
print(f"   MED:   {len(chain_df[chain_df['bucket_type'] == 'MED'])}")
print(f"   SHORT: {len(chain_df[chain_df['bucket_type'] == 'SHORT'])}")


# ── Score all options ──
print("\n🎯 Scoring options with 6-factor edge model…\n")

scored_rows = []
for _, row in chain_df.iterrows():
    edge, p_profit, components = score_option(row, hv_momentum, STYLE_PREFS)
    if np.isnan(edge):
        continue

    # Assign DTE bucket
    dte = row["dte"]
    if dte >= 300:
        bucket = "🏗️ 300+ DTE"
    elif dte >= 30:
        bucket = "🔄 30-90 DTE"
    else:
        bucket = "⚡ <30 DTE"

    # Bounce score for this ticker
    b_match = bounce_df[bounce_df["Ticker"] == row["ticker"]]
    b_score = float(b_match["Bounce Score"].iloc[0]) if not b_match.empty else 0

    # R/R at +10%
    upside_spot = row["spot"] * 1.10
    payout = max(upside_spot - row["strike"], 0)
    rr = (payout - row["mid"]) / row["mid"] if row["mid"] > 0 else 0

    scored_rows.append(
        {
            "Ticker": row["ticker"],
            "Sector": sector_map.get(row["ticker"], "—"),
            "Bucket": bucket,
            "Expiry": row["expiration"],
            "DTE": dte,
            "Strike": row["strike"],
            "Moneyness": row["moneyness"],
            "Spot": row["spot"],
            "Mid": row["mid"],
            "IV": row.get("iv", np.nan),
            "HV30": hv_momentum.get(row["ticker"], {}).get("hv_30", np.nan),
            "Breakeven %": row["breakeven_pct"],
            "Leverage": row["leverage"],
            "Cost/Contract": row["cost_per_contract"],
            "OI": safe_float(row.get("openInterest", 0)),
            "Spread %": row.get("spread_pct", np.nan),
            "P(Profit)": p_profit,
            "Edge Score": edge,
            "Bounce Score": b_score,
            "R/R": rr,
            "Val Score": val_scores.get(row["ticker"], 50),
            "s_pprofit": components.get("p_profit", np.nan),
            "s_momentum": components.get("momentum", np.nan),
            "s_breakeven": components.get("breakeven", np.nan),
            "s_iv_edge": components.get("iv_edge", np.nan),
            "s_style": components.get("style_match", np.nan),
            "s_liquidity": components.get("liquidity", np.nan),
            "Ret 1M": hv_momentum.get(row["ticker"], {}).get("ret_1m", np.nan),
            "Ret 3M": hv_momentum.get(row["ticker"], {}).get("ret_3m", np.nan),
            "RSI": hv_momentum.get(row["ticker"], {}).get("rsi", np.nan),
        }
    )

scored_df = pd.DataFrame(scored_rows)
print(
    f"✅ Scored {len(scored_df)} options across {scored_df['Ticker'].nunique()} tickers"
)
print(f"   By bucket: {scored_df.groupby('Bucket').size().to_dict()}")

# ── Display: Top Scored LEAPS ──
display(Markdown("### 🏗️ Top 20 Scored LEAPS"))

leaps_top = (
    scored_df[scored_df["Bucket"] == "🏗️ 300+ DTE"]
    .sort_values("Edge Score", ascending=False)
    .head(20)
)
show_cols = [
    "Ticker",
    "Sector",
    "Expiry",
    "DTE",
    "Strike",
    "Moneyness",
    "Mid",
    "IV",
    "Breakeven %",
    "Leverage",
    "P(Profit)",
    "Edge Score",
    "Bounce Score",
    "R/R",
    "Val Score",
]
sa = [c for c in show_cols if c in leaps_top.columns]
display(
    leaps_top[sa]
    .style.format(
        {
            "Strike": "${:,.0f}",
            "Mid": "${:,.2f}",
            "Moneyness": "{:.0%}",
            "IV": "{:.0%}",
            "Breakeven %": "{:+.1%}",
            "Leverage": "{:.0f}x",
            "P(Profit)": "{:.0%}",
            "Edge Score": "{:.0f}",
            "Bounce Score": "{:.0f}",
            "R/R": "{:.1f}x",
            "Val Score": "{:.0f}",
        },
        na_rep="—",
    )
    .background_gradient(subset=["Edge Score"], cmap="RdYlGn", vmin=30, vmax=70)
    .background_gradient(subset=["Val Score"], cmap="RdYlGn", vmin=30, vmax=80)
    .set_caption("Top LEAPS by Edge Score")
)


# ── IV Smile Charts for Top 5 LEAPS Tickers ──
display(Markdown("### 😊 IV Smile — Top 5 LEAPS Tickers"))

top5_leaps_tickers = (
    leaps_top.groupby("Ticker")["Edge Score"].max().nlargest(5).index.tolist()
)
if top5_leaps_tickers:
    fig_smile = make_subplots(rows=1, cols=1)
    for ticker in top5_leaps_tickers:
        t_data = scored_df[
            (scored_df["Ticker"] == ticker) & (scored_df["Bucket"] == "🏗️ 300+ DTE")
        ].sort_values("Strike")
        if len(t_data) > 2 and t_data["IV"].notna().sum() > 2:
            fig_smile.add_trace(
                go.Scatter(
                    x=t_data["Moneyness"],
                    y=t_data["IV"],
                    mode="lines+markers",
                    name=ticker,
                    hovertemplate=f"{ticker}<br>Strike: $%{{customdata[0]:,.0f}}<br>IV: %{{y:.1%}}<br>Moneyness: %{{x:.0%}}",
                    customdata=t_data[["Strike"]].values,
                )
            )
    fig_smile.update_layout(
        title="IV Smile — LEAPS Chains (Top 5 by Edge Score)",
        xaxis_title="Moneyness (Strike / Spot)",
        yaxis_title="Implied Volatility",
        height=450,
        xaxis_tickformat=".0%",
        yaxis_tickformat=".0%",
    )
    fig_smile.add_vline(
        x=1.0, line_dash="dot", line_color="gray", annotation_text="ATM"
    )
    fig_smile.show()

---

## 🎨 Step 5 — Diversity Enforcement & Composite Ranking

Build a correlation matrix from 6-month daily returns. Apply:

- **Sector caps** — max 3 tickers per sector (prevent tech overload)
- **Correlation penalty** — reduce scores for ρ > 0.65 pairs
- **Beta spread** — reward portfolio-level beta balance

Composite Final Score = Edge + Valuation + Ratios + Bounce + Diversity bonuses.
Auto-select top picks per DTE bucket. Visualise with sector treemap + correlation heatmap.


In [ ]:
# ══════════════════════════════════════════════════════════════════
#  🎨  STEP 5: Diversity Enforcement & Composite Ranking
# ══════════════════════════════════════════════════════════════════

# ── 5A: Correlation Matrix ──
print("Building correlation matrix (6-month daily returns)…\n")

returns_dict = {}
for ticker in UNIVERSE:
    try:
        t = yf.Ticker(ticker)
        time.sleep(RATE_LIMIT_SLEEP)
        hist = t.history(period="6mo")
        if not hist.empty and len(hist) > 30:
            returns_dict[ticker] = hist["Close"].pct_change().dropna()
    except Exception:
        pass

returns_matrix = pd.DataFrame(returns_dict)
corr_matrix = returns_matrix.corr()
print(f"✅ Correlation matrix: {corr_matrix.shape[0]}×{corr_matrix.shape[1]}")

# Display heatmap
display(Markdown("### 🔗 Universe Correlation Heatmap"))
if corr_matrix.shape[0] > 1:
    fig_corr = go.Figure(
        data=go.Heatmap(
            z=corr_matrix.values,
            x=corr_matrix.columns.tolist(),
            y=corr_matrix.index.tolist(),
            colorscale="RdYlGn_r",
            zmin=-0.2,
            zmax=1.0,
            text=np.round(corr_matrix.values, 2),
            texttemplate="%{text}",
            textfont={"size": 9},
        )
    )
    fig_corr.update_layout(
        title="Pairwise Correlation (6M daily returns)",
        height=max(400, len(corr_matrix) * 22),
        width=max(500, len(corr_matrix) * 22),
    )
    fig_corr.show()


# ── 5B: Merge ratios onto scored_df → unified_df ──
ratio_df = (
    pd.DataFrame(ticker_ratios).T.reset_index().rename(columns={"index": "Ticker"})
)
ratio_df = ratio_df.rename(
    columns={
        "sharpe": "Sharpe",
        "sortino": "Sortino",
        "calmar": "Calmar",
        "max_dd": "Max DD",
        "win_rate": "Win Rate",
        "ann_ret": "Ann Ret",
        "ann_vol": "Ann Vol",
        "beta": "Beta",
    }
)

# Drop old cols if re-running
for c in [
    "Sharpe",
    "Sortino",
    "Calmar",
    "Max DD",
    "Win Rate",
    "Ann Ret",
    "Ann Vol",
    "Beta",
]:
    if c in scored_df.columns:
        scored_df = scored_df.drop(columns=[c])

unified_df = scored_df.merge(
    ratio_df[
        [
            "Ticker",
            "Sharpe",
            "Sortino",
            "Calmar",
            "Max DD",
            "Win Rate",
            "Ann Ret",
            "Ann Vol",
            "Beta",
        ]
    ],
    on="Ticker",
    how="left",
)


# ── 5C: Combo Score → Final Score → Portfolio Score ──


# Combo Score (Edge + Bounce weighted)
def combo_score(row):
    edge = row.get("Edge Score", 0)
    bounce = row.get("Bounce Score", 0)
    if np.isnan(edge):
        edge = 0
    return edge * 0.70 + bounce * 0.30


unified_df["Combo Score"] = unified_df.apply(combo_score, axis=1)


# Final Score — bucket-specific weighting
def final_score(row):
    combo = row["Combo Score"] if not np.isnan(row["Combo Score"]) else 0
    sharpe = row.get("Sharpe", 0) if not np.isnan(row.get("Sharpe", np.nan)) else 0
    sortino = row.get("Sortino", 0) if not np.isnan(row.get("Sortino", np.nan)) else 0
    rr = row.get("R/R", 0) if not np.isnan(row.get("R/R", np.nan)) else 0
    win = row.get("Win Rate", 0.5) if not np.isnan(row.get("Win Rate", np.nan)) else 0.5
    calmar = row.get("Calmar", 0) if not np.isnan(row.get("Calmar", np.nan)) else 0

    sharpe_n = np.clip((sharpe + 1) * 33, 0, 100)
    sortino_n = np.clip((sortino + 1) * 25, 0, 100)
    rr_n = np.clip(rr * 20, 0, 100)
    win_n = np.clip(win * 100 - 20, 0, 100)
    calmar_n = np.clip(calmar * 25, 0, 100)

    bucket = row["Bucket"]
    if bucket == "⚡ <30 DTE":
        return combo * 0.40 + rr_n * 0.25 + sharpe_n * 0.15 + win_n * 0.20
    elif bucket == "🔄 30-90 DTE":
        return (
            combo * 0.35
            + sharpe_n * 0.20
            + sortino_n * 0.15
            + rr_n * 0.15
            + win_n * 0.15
        )
    else:
        return (
            combo * 0.30
            + sortino_n * 0.20
            + calmar_n * 0.15
            + sharpe_n * 0.15
            + rr_n * 0.10
            + win_n * 0.10
        )


unified_df["Final Score"] = unified_df.apply(final_score, axis=1)


# Portfolio Score — add valuation
def portfolio_score(row):
    fs = row.get("Final Score", 0)
    if np.isnan(fs):
        fs = 0
    vs = row.get("Val Score", 50)
    bucket = row["Bucket"]
    if bucket == "⚡ <30 DTE":
        return fs * 0.85 + vs * 0.15
    elif bucket == "🔄 30-90 DTE":
        return fs * 0.80 + vs * 0.20
    else:
        return fs * 0.70 + vs * 0.30


unified_df["Portfolio Score"] = unified_df.apply(portfolio_score, axis=1)

print(
    f"✅ Unified DataFrame: {len(unified_df)} options, {unified_df['Ticker'].nunique()} tickers"
)
print(
    f"   Score range: {unified_df['Portfolio Score'].min():.0f} – {unified_df['Portfolio Score'].max():.0f}"
)


# ── 5D: Sector diversity check ──
display(Markdown("### 🏭 Sector Diversity — Universe"))

sector_df = pd.DataFrame(
    [
        {
            "Ticker": t,
            "Sector": sector_map.get(t, "—"),
            "β": beta_map.get(t, 1.0),
            "Val Score": val_scores.get(t, 50),
            "Best Edge": unified_df[unified_df["Ticker"] == t]["Edge Score"].max()
            if t in unified_df["Ticker"].values
            else 0,
            "Bounce": bounce_df[bounce_df["Ticker"] == t]["Bounce Score"].iloc[0]
            if t in bounce_df["Ticker"].values
            else 0,
            "Sharpe": ticker_ratios.get(t, {}).get("sharpe", 0),
            "# Options": len(unified_df[unified_df["Ticker"] == t]),
        }
        for t in UNIVERSE
    ]
).sort_values("Best Edge", ascending=False)

display(
    sector_df.style.format(
        {
            "β": "{:.2f}",
            "Val Score": "{:.0f}",
            "Best Edge": "{:.0f}",
            "Bounce": "{:.0f}",
            "Sharpe": "{:+.2f}",
            "# Options": "{:.0f}",
        },
        na_rep="—",
    )
    .background_gradient(subset=["Best Edge"], cmap="RdYlGn", vmin=20, vmax=60)
    .set_caption(
        f"Universe: {len(UNIVERSE)} tickers across {sector_df['Sector'].nunique()} sectors"
    )
)


# ── Sector Treemap ──
if len(sector_df) > 0:
    tree_df = sector_df.copy()
    tree_df["Size"] = tree_df["# Options"].clip(lower=1)
    fig_tree = px.treemap(
        tree_df,
        path=["Sector", "Ticker"],
        values="Size",
        color="Best Edge",
        color_continuous_scale="RdYlGn",
        color_continuous_midpoint=40,
        title="Universe by Sector (size = # options, color = Edge Score)",
    )
    fig_tree.update_layout(height=500)
    fig_tree.show()


# ── 5E: Auto-Select Picks per Bucket ──
display(Markdown("---\n### 🏆 Auto-Selected Picks by Bucket"))

PICKS_PER_BUCKET = 5

bucket_configs = [
    (
        "⚡ <30 DTE",
        "⚡ Short-Term Swings",
        "Quick directional bets — high leverage, tight risk. R/R + win rate.",
    ),
    (
        "🔄 30-90 DTE",
        "🔄 Medium-Term Recovery",
        "1-3 month horizon for oversold reversals. Sharpe + Sortino.",
    ),
    (
        "🏗️ 300+ DTE",
        "🏗️ LEAPS — Conviction Holds",
        "Stock replacement, deep ITM. Sortino + Calmar + valuation.",
    ),
]

all_auto_picks = []
for bucket_key, title, desc in bucket_configs:
    sub = unified_df[unified_df["Bucket"] == bucket_key].copy()
    if sub.empty:
        display(Markdown(f"\n### {title}\n*No options in this bucket.*"))
        continue

    # Pick best 1 per ticker, enforce sector cap
    best_per_ticker = (
        sub.sort_values("Portfolio Score", ascending=False).groupby("Ticker").head(1)
    )
    # Sector cap
    sector_counts_sel = {}
    filtered_rows = []
    for _, r in best_per_ticker.sort_values(
        "Portfolio Score", ascending=False
    ).iterrows():
        sec = r.get("Sector", "Unknown")
        if sector_counts_sel.get(sec, 0) >= MAX_PER_SECTOR:
            continue
        sector_counts_sel[sec] = sector_counts_sel.get(sec, 0) + 1
        filtered_rows.append(r)
        if len(filtered_rows) >= PICKS_PER_BUCKET:
            break
    top = pd.DataFrame(filtered_rows)

    display(Markdown(f"\n### {title}"))
    display(Markdown(f"*{desc}*"))

    if not top.empty:
        pick_cols = [
            "Ticker",
            "Sector",
            "Expiry",
            "DTE",
            "Strike",
            "Moneyness",
            "Mid",
            "IV",
            "Breakeven %",
            "Leverage",
            "P(Profit)",
            "Sharpe",
            "Sortino",
            "R/R",
            "Edge Score",
            "Bounce Score",
            "Val Score",
            "Portfolio Score",
        ]
        pc = [c for c in pick_cols if c in top.columns]
        display(
            top[pc]
            .style.format(
                {
                    "Strike": "${:,.0f}",
                    "Mid": "${:,.2f}",
                    "Moneyness": "{:.0%}",
                    "IV": "{:.0%}",
                    "Breakeven %": "{:+.1%}",
                    "Leverage": "{:.0f}x",
                    "P(Profit)": "{:.0%}",
                    "Sharpe": "{:+.2f}",
                    "Sortino": "{:+.2f}",
                    "R/R": "{:.1f}x",
                    "Edge Score": "{:.0f}",
                    "Bounce Score": "{:.0f}",
                    "Val Score": "{:.0f}",
                    "Portfolio Score": "{:.0f}",
                },
                na_rep="—",
            )
            .background_gradient(
                subset=["Portfolio Score"], cmap="RdYlGn", vmin=25, vmax=65
            )
            .set_caption(f"{title} — Top {PICKS_PER_BUCKET} picks")
        )

    all_auto_picks.append(top)

# ── Summary: All Picks ──
if all_auto_picks:
    all_picks_df = pd.concat(all_auto_picks, ignore_index=True)
    display(Markdown("\n---\n### 📋 All Auto-Picks at a Glance"))
    sum_cols = [
        "Bucket",
        "Ticker",
        "Sector",
        "Expiry",
        "DTE",
        "Strike",
        "Moneyness",
        "Mid",
        "P(Profit)",
        "Sharpe",
        "R/R",
        "Edge Score",
        "Val Score",
        "Portfolio Score",
    ]
    sc = [c for c in sum_cols if c in all_picks_df.columns]
    display(
        all_picks_df[sc]
        .style.format(
            {
                "Strike": "${:,.0f}",
                "Mid": "${:,.2f}",
                "Moneyness": "{:.0%}",
                "P(Profit)": "{:.0%}",
                "Sharpe": "{:+.2f}",
                "R/R": "{:.1f}x",
                "Edge Score": "{:.0f}",
                "Val Score": "{:.0f}",
                "Portfolio Score": "{:.0f}",
            },
            na_rep="—",
        )
        .background_gradient(
            subset=["Portfolio Score"], cmap="RdYlGn", vmin=25, vmax=65
        )
        .set_caption("All auto-picks — sector-diverse, ranked by Portfolio Score")
    )

    # Ratio chart for picked tickers
    picked_tickers = all_picks_df["Ticker"].unique().tolist()
    rp = ratio_df[ratio_df["Ticker"].isin(picked_tickers)].copy()
    if not rp.empty:
        fig_r = go.Figure()
        fig_r.add_trace(
            go.Bar(
                name="Sharpe", x=rp["Ticker"], y=rp["Sharpe"], marker_color="#3498db"
            )
        )
        fig_r.add_trace(
            go.Bar(
                name="Sortino", x=rp["Ticker"], y=rp["Sortino"], marker_color="#2ecc71"
            )
        )
        fig_r.add_trace(
            go.Bar(
                name="Calmar", x=rp["Ticker"], y=rp["Calmar"], marker_color="#e67e22"
            )
        )
        fig_r.update_layout(
            barmode="group",
            title="Risk-Adjusted Ratios — Picked Tickers",
            height=400,
            yaxis_title="Ratio Value",
        )
        fig_r.add_hline(y=0, line_dash="dot", line_color="gray")
        fig_r.show()

    # Scatter: Portfolio Score vs Sharpe
    fig_sc = px.scatter(
        all_picks_df,
        x="Sharpe",
        y="Portfolio Score",
        text="Ticker",
        color="Bucket",
        size="Leverage",
        size_max=25,
        color_discrete_map={
            "⚡ <30 DTE": "#e74c3c",
            "🔄 30-90 DTE": "#f39c12",
            "🏗️ 300+ DTE": "#2ecc71",
        },
        title="Portfolio Score vs Sharpe (size = leverage)",
    )
    fig_sc.update_traces(textposition="top center")
    fig_sc.show()

---

## 📦 Step 6: Portfolio Construction & Final Dashboard

Round-robin allocation across DTE buckets, Kelly-fractional sizing, full risk dashboard with pie charts, scenario P&L, correlation heatmap, and portfolio summary.


In [ ]:
# ══════════════════════════════════════════════════════════════════
#  📦  STEP 6: Portfolio Construction & Dashboard
# ══════════════════════════════════════════════════════════════════

display(Markdown("# 📦 Portfolio Construction"))
display(Markdown(f"**Starting capital:** ${STARTING_BALANCE:,.0f}"))


# ── 6A: Kelly Fraction ──
def kelly_fraction(
    p_profit,
    reward_to_risk,
    fractional=FRACTIONAL_KELLY,
    cap=KELLY_CAP,
    haircut=EDGE_HAIRCUT,
):
    """Half-Kelly with edge haircut."""
    if np.isnan(p_profit) or np.isnan(reward_to_risk) or reward_to_risk <= 0:
        return 0.02
    p = p_profit * haircut
    q = 1 - p
    b = reward_to_risk
    f = (p * b - q) / b
    return float(np.clip(f * fractional, 0.02, cap))


# ── 6B: Build candidate pool from all_picks_df ──
if all_auto_picks:
    pool = pd.concat(all_auto_picks, ignore_index=True).copy()
else:
    pool = (
        unified_df.sort_values("Portfolio Score", ascending=False)
        .groupby("Ticker")
        .head(1)
        .head(15)
        .copy()
    )

# Ensure required columns
if "contracts" not in pool.columns:
    pool["contracts"] = 1

print(
    f"📋 Candidate pool: {len(pool)} options across {pool['Ticker'].nunique()} tickers"
)


# ── 6C: Round-Robin Bucket Builder ──


def try_add_position(
    row, remaining_cash, positions, max_corr_with_existing=CORRELATION_PENALTY_THRESH
):
    """Try to add a position. Returns (success, cost, position_dict)."""
    mid = row.get("Mid", np.nan)
    if np.isnan(mid) or mid <= 0:
        return False, 0, {}
    cost_1 = mid * 100  # 1 contract

    # Kelly sizing
    p_profit = row.get("P(Profit)", 0.5)
    rr = row.get("R/R", 1.0)
    kf = kelly_fraction(p_profit, rr)
    kelly_alloc = STARTING_BALANCE * kf
    contracts = max(1, int(kelly_alloc / cost_1))

    total_cost = contracts * cost_1
    if total_cost > remaining_cash:
        contracts = max(1, int(remaining_cash / cost_1))
        total_cost = contracts * cost_1
    if total_cost > remaining_cash or total_cost < 50:
        return False, 0, {}

    # Correlation check with existing positions
    ticker = row["Ticker"]
    for pos in positions:
        pt = pos["Ticker"]
        if ticker == pt:
            return False, 0, {}  # already have this ticker
        if ticker in corr_matrix.columns and pt in corr_matrix.columns:
            c = corr_matrix.loc[ticker, pt]
            if not np.isnan(c) and c > max_corr_with_existing:
                # Penalise but don't block — reduce contracts
                contracts = max(1, contracts - 1)
                total_cost = contracts * cost_1
                if total_cost > remaining_cash:
                    return False, 0, {}

    pos_dict = {
        "Ticker": ticker,
        "Sector": row.get("Sector", "—"),
        "Bucket": row["Bucket"],
        "Expiry": row.get("Expiry", "—"),
        "DTE": row.get("DTE", 0),
        "Strike": row.get("Strike", 0),
        "Moneyness": row.get("Moneyness", 1.0),
        "Mid": mid,
        "IV": row.get("IV", np.nan),
        "Breakeven %": row.get("Breakeven %", np.nan),
        "Leverage": row.get("Leverage", 1),
        "P(Profit)": p_profit,
        "Sharpe": row.get("Sharpe", 0),
        "Sortino": row.get("Sortino", 0),
        "R/R": rr,
        "Edge Score": row.get("Edge Score", 0),
        "Bounce Score": row.get("Bounce Score", 0),
        "Val Score": row.get("Val Score", 50),
        "Portfolio Score": row.get("Portfolio Score", 0),
        "Kelly %": kf,
        "Contracts": contracts,
        "Total Cost": total_cost,
    }
    return True, total_cost, pos_dict


# Round-robin across buckets
bucket_order = ["🏗️ 300+ DTE", "🔄 30-90 DTE", "⚡ <30 DTE"]
positions = []
remaining = float(STARTING_BALANCE)
MAX_ROUNDS = 6
MAX_PER_TICKER = 1

print(f"\n🔄 Starting round-robin allocation (max {MAX_ROUNDS} rounds)…\n")

for rd in range(MAX_ROUNDS):
    added_this_round = False
    for bucket in bucket_order:
        bucket_pool = pool[pool["Bucket"] == bucket].sort_values(
            "Portfolio Score", ascending=False
        )
        for _, row in bucket_pool.iterrows():
            # Skip if already in portfolio
            if row["Ticker"] in [
                p["Ticker"] for p in positions if p["Bucket"] == bucket
            ]:
                continue
            # Sector cap in portfolio
            sector_in_port = sum(
                1 for p in positions if p["Sector"] == row.get("Sector", "—")
            )
            if sector_in_port >= MAX_PER_SECTOR:
                continue

            ok, cost, pos = try_add_position(row, remaining, positions)
            if ok:
                positions.append(pos)
                remaining -= cost
                added_this_round = True
                print(
                    f"  ✅ R{rd + 1} | {bucket} | {pos['Ticker']:5s} | {pos['Contracts']}× "
                    f"${pos['Strike']:.0f} {pos['Expiry']} | ${cost:,.0f}"
                )
                break  # next bucket

    if not added_this_round:
        print(f"  ⏹️ No more positions added in round {rd + 1}, stopping.")
        break

portfolio_df = pd.DataFrame(positions)

deployed = portfolio_df["Total Cost"].sum() if not portfolio_df.empty else 0
cash_left = STARTING_BALANCE - deployed
deploy_pct = deployed / STARTING_BALANCE * 100

print(f"\n{'=' * 50}")
print(f"💰 Deployed: ${deployed:,.0f} ({deploy_pct:.0f}%)")
print(f"💵 Cash reserve: ${cash_left:,.0f} ({100 - deploy_pct:.0f}%)")
print(
    f"📊 Positions: {len(positions)} across {portfolio_df['Ticker'].nunique()} tickers"
)
print(f"{'=' * 50}")


# ══════════════════════════════════════════════════════════════════
#  📊 DASHBOARD
# ══════════════════════════════════════════════════════════════════
if not portfolio_df.empty:
    display(Markdown("---\n## 📊 Full Portfolio Dashboard"))

    # ── Holdings Table ──
    display(Markdown("### 📋 Holdings"))
    h_cols = [
        "Ticker",
        "Sector",
        "Bucket",
        "Expiry",
        "DTE",
        "Strike",
        "Moneyness",
        "Mid",
        "Contracts",
        "Total Cost",
        "IV",
        "P(Profit)",
        "Leverage",
        "Sharpe",
        "Sortino",
        "R/R",
        "Edge Score",
        "Portfolio Score",
        "Kelly %",
    ]
    hc = [c for c in h_cols if c in portfolio_df.columns]
    display(
        portfolio_df[hc]
        .style.format(
            {
                "Strike": "${:,.0f}",
                "Mid": "${:,.2f}",
                "Moneyness": "{:.0%}",
                "Total Cost": "${:,.0f}",
                "IV": "{:.0%}",
                "P(Profit)": "{:.0%}",
                "Leverage": "{:.0f}x",
                "Sharpe": "{:+.2f}",
                "Sortino": "{:+.2f}",
                "R/R": "{:.1f}x",
                "Edge Score": "{:.0f}",
                "Portfolio Score": "{:.0f}",
                "Kelly %": "{:.1%}",
            },
            na_rep="—",
        )
        .background_gradient(
            subset=["Portfolio Score"], cmap="RdYlGn", vmin=20, vmax=65
        )
        .background_gradient(subset=["P(Profit)"], cmap="RdYlGn", vmin=0.3, vmax=0.7)
        .set_caption(
            f"Portfolio: {len(portfolio_df)} positions, ${deployed:,.0f} deployed"
        )
    )

    # ── Risk Summary ──
    display(Markdown("### ⚠️ Risk Summary"))

    w = portfolio_df["Total Cost"] / deployed
    hhi = (w**2).sum()
    wp = (portfolio_df["P(Profit)"] * w).sum()
    wrr = (portfolio_df["R/R"] * w).sum()
    wdte = (portfolio_df["DTE"] * w).sum()
    wlev = (portfolio_df["Leverage"] * w).sum()
    w_sharpe = (portfolio_df["Sharpe"] * w).sum()
    w_sortino = (portfolio_df["Sortino"] * w).sum()

    # Corr among portfolio tickers
    port_tickers = portfolio_df["Ticker"].unique().tolist()
    port_corr = (
        corr_matrix.loc[
            [t for t in port_tickers if t in corr_matrix.columns],
            [t for t in port_tickers if t in corr_matrix.columns],
        ]
        if len(port_tickers) > 1
        else pd.DataFrame()
    )
    avg_corr = (
        port_corr.values[np.triu_indices_from(port_corr, k=1)].mean()
        if port_corr.shape[0] > 1
        else 0
    )

    risk_md = f"""
| Metric | Value |
|--------|-------|
| 💰 Deployed | ${deployed:,.0f} ({deploy_pct:.0f}%) |
| 💵 Cash Reserve | ${cash_left:,.0f} |
| 📊 Positions | {len(portfolio_df)} across {portfolio_df["Ticker"].nunique()} tickers |
| 🏭 Sectors | {portfolio_df["Sector"].nunique()} |
| 📐 HHI (concentration) | {hhi:.3f} {"✅ diversified" if hhi < 0.20 else "⚠️ concentrated"} |
| 🎯 Wtd P(Profit) | {wp:.0%} |
| ⚖️ Wtd R/R | {wrr:.1f}x |
| 📈 Wtd Sharpe | {w_sharpe:+.2f} |
| 📊 Wtd Sortino | {w_sortino:+.2f} |
| 🔗 Avg Pairwise Corr | {avg_corr:.2f} |
| ⏳ Wtd DTE | {wdte:.0f} days |
| 🔧 Wtd Leverage | {wlev:.0f}x |
"""
    display(Markdown(risk_md))

    # ── Pie Charts ──
    display(Markdown("### 🥧 Allocation Breakdown"))
    fig_pies = make_subplots(
        rows=1,
        cols=3,
        specs=[[{"type": "pie"}] * 3],
        subplot_titles=["By Bucket", "By Ticker", "By Sector"],
    )

    bucket_alloc = portfolio_df.groupby("Bucket")["Total Cost"].sum()
    fig_pies.add_trace(
        go.Pie(
            labels=bucket_alloc.index.tolist(),
            values=bucket_alloc.values,
            hole=0.4,
            textinfo="label+percent",
            marker=dict(colors=["#2ecc71", "#f39c12", "#e74c3c"]),
        ),
        1,
        1,
    )

    ticker_alloc = portfolio_df.groupby("Ticker")["Total Cost"].sum()
    fig_pies.add_trace(
        go.Pie(
            labels=ticker_alloc.index.tolist(),
            values=ticker_alloc.values,
            hole=0.4,
            textinfo="label+percent",
        ),
        1,
        2,
    )

    sector_alloc = portfolio_df.groupby("Sector")["Total Cost"].sum()
    fig_pies.add_trace(
        go.Pie(
            labels=sector_alloc.index.tolist(),
            values=sector_alloc.values,
            hole=0.4,
            textinfo="label+percent",
        ),
        1,
        3,
    )

    fig_pies.update_layout(height=400, title_text="Portfolio Allocation")
    fig_pies.show()

    # ── Bucket vs Target ──
    display(Markdown("### 🎯 Bucket Allocation vs Target"))
    actual_pcts = {}
    for bkt, target in BUCKET_TARGET.items():
        cost = bucket_alloc.get(bkt, 0)
        actual_pcts[bkt] = {
            "Target %": target * 100,
            "Actual %": cost / deployed * 100 if deployed > 0 else 0,
            "Actual $": cost,
            "Δ": (cost / deployed * 100 - target * 100) if deployed > 0 else 0,
        }
    bucket_comp = pd.DataFrame(actual_pcts).T
    display(
        bucket_comp.style.format(
            {
                "Target %": "{:.0f}%",
                "Actual %": "{:.0f}%",
                "Actual $": "${:,.0f}",
                "Δ": "{:+.1f}pp",
            }
        ).set_caption("Bucket Allocation vs Target")
    )

    # ── Scenario P&L Waterfall ──
    display(Markdown("### 📈 Scenario P&L (stock move → portfolio impact)"))

    scenarios = [-0.20, -0.15, -0.10, -0.05, 0.0, 0.05, 0.10, 0.15, 0.20, 0.30]
    port_pnl_by_scenario = {}

    for s in scenarios:
        total_pnl = 0
        for _, row in portfolio_df.iterrows():
            spot = spot_map.get(row["Ticker"], row["Strike"])
            new_spot = spot * (1 + s)
            strike = row["Strike"]
            contracts = row["Contracts"]
            cost = row["Total Cost"]
            intrinsic = max(0, new_spot - strike) * contracts * 100
            pnl = intrinsic - cost
            total_pnl += pnl
        port_pnl_by_scenario[f"{s:+.0%}"] = total_pnl

    pnl_df = pd.DataFrame(list(port_pnl_by_scenario.items()), columns=["Move", "P&L"])
    colors = ["#e74c3c" if v < 0 else "#2ecc71" for v in pnl_df["P&L"]]

    fig_pnl = go.Figure(
        go.Bar(
            x=pnl_df["Move"],
            y=pnl_df["P&L"],
            marker_color=colors,
            text=[f"${v:,.0f}" for v in pnl_df["P&L"]],
            textposition="outside",
        )
    )
    fig_pnl.add_hline(y=0, line_dash="dot", line_color="gray")
    fig_pnl.add_hline(
        y=-deployed,
        line_dash="dash",
        line_color="red",
        annotation_text="Max Loss",
        annotation_position="bottom left",
    )
    fig_pnl.update_layout(
        title="Portfolio P&L at Expiry by Stock Move Scenario",
        xaxis_title="Underlying Move",
        yaxis_title="P&L ($)",
        height=450,
    )
    fig_pnl.show()

    # ── Per-Position Return Matrix ──
    display(Markdown("### 📊 Per-Position Return Matrix"))
    return_matrix = []
    for _, row in portfolio_df.iterrows():
        spot = spot_map.get(row["Ticker"], row["Strike"])
        ret_row = {
            "Ticker": row["Ticker"],
            "Bucket": row["Bucket"],
            "Cost": row["Total Cost"],
        }
        for s in [-0.10, -0.05, 0.0, 0.05, 0.10, 0.20]:
            new_spot = spot * (1 + s)
            intrinsic = max(0, new_spot - row["Strike"]) * row["Contracts"] * 100
            pnl = intrinsic - row["Total Cost"]
            ret_pct = pnl / row["Total Cost"] * 100 if row["Total Cost"] > 0 else 0
            ret_row[f"{s:+.0%}"] = ret_pct
        return_matrix.append(ret_row)

    ret_df = pd.DataFrame(return_matrix)
    ret_cols = [f"{s:+.0%}" for s in [-0.10, -0.05, 0.0, 0.05, 0.10, 0.20]]
    display(
        ret_df.style.format({c: "{:+.0f}%" for c in ret_cols} | {"Cost": "${:,.0f}"})
        .background_gradient(subset=ret_cols, cmap="RdYlGn", vmin=-100, vmax=100)
        .set_caption("Return (%) by scenario at expiry")
    )

    # ── Portfolio Correlation Heatmap ──
    if len(port_tickers) > 1 and port_corr.shape[0] > 1:
        display(Markdown("### 🔗 Portfolio Correlation Heatmap"))
        fig_pc = go.Figure(
            data=go.Heatmap(
                z=port_corr.values,
                x=port_corr.columns.tolist(),
                y=port_corr.index.tolist(),
                colorscale="RdYlGn_r",
                zmin=-0.2,
                zmax=1.0,
                text=np.round(port_corr.values, 2),
                texttemplate="%{text}",
            )
        )
        fig_pc.update_layout(
            title="Correlation — Portfolio Holdings", height=400, width=500
        )
        fig_pc.show()

    # ── Sizing Breakdown ──
    display(Markdown("### 📐 Position Sizing Breakdown"))

    sizing_df = portfolio_df[
        [
            "Ticker",
            "Bucket",
            "Contracts",
            "Mid",
            "Total Cost",
            "Kelly %",
            "P(Profit)",
            "R/R",
        ]
    ].copy()
    sizing_df["Weight %"] = sizing_df["Total Cost"] / deployed * 100
    sizing_df["Max Loss"] = sizing_df["Total Cost"]
    display(
        sizing_df.style.format(
            {
                "Mid": "${:,.2f}",
                "Total Cost": "${:,.0f}",
                "Kelly %": "{:.1%}",
                "P(Profit)": "{:.0%}",
                "R/R": "{:.1f}x",
                "Weight %": "{:.1f}%",
                "Max Loss": "${:,.0f}",
            }
        )
        .background_gradient(subset=["Weight %"], cmap="Blues")
        .set_caption("Sizing: Kelly-fractional with correlation penalty")
    )

    # ══════════════════════════════════════════════════════════════
    #  ✅ FINAL SUMMARY
    # ══════════════════════════════════════════════════════════════
    display(Markdown("---"))
    display(Markdown("## ✅ Portfolio Summary"))

    leaps_pct = (
        bucket_alloc.get("🏗️ 300+ DTE", 0) / deployed * 100 if deployed > 0 else 0
    )
    med_pct = (
        bucket_alloc.get("🔄 30-90 DTE", 0) / deployed * 100 if deployed > 0 else 0
    )
    short_pct = (
        bucket_alloc.get("⚡ <30 DTE", 0) / deployed * 100 if deployed > 0 else 0
    )

    final_md = f"""
### 🏁 Final Portfolio — ${deployed:,.0f} / ${STARTING_BALANCE:,.0f} deployed ({deploy_pct:.0f}%)

| Attribute | Value |
|-----------|-------|
| Positions | **{len(portfolio_df)}** across **{portfolio_df["Ticker"].nunique()}** tickers |
| Sectors   | **{portfolio_df["Sector"].nunique()}** |
| 🏗️ LEAPS  | {leaps_pct:.0f}% (target {BUCKET_TARGET.get("🏗️ 300+ DTE", 0) * 100:.0f}%) |
| 🔄 Medium | {med_pct:.0f}% (target {BUCKET_TARGET.get("🔄 30-90 DTE", 0) * 100:.0f}%) |
| ⚡ Short   | {short_pct:.0f}% (target {BUCKET_TARGET.get("⚡ <30 DTE", 0) * 100:.0f}%) |
| HHI       | {hhi:.3f} |
| Avg Corr  | {avg_corr:.2f} |
| Wtd P(Profit) | {wp:.0%} |
| Wtd R/R   | {wrr:.1f}x |
| Wtd Sharpe | {w_sharpe:+.2f} |
| Wtd Sortino | {w_sortino:+.2f} |
| Cash Left | ${cash_left:,.0f} |

> ⚠️ **Not financial advice.** This is a research-grade screening notebook.
> Always verify with your own due diligence before trading.
"""
    display(Markdown(final_md))

else:
    display(
        Markdown("⚠️ **No positions could be built.** Check candidate pool and balance.")
    )

## 🎯 Step 7 — Screener Output & Watchlist

Quick-audit flags for the top picks, then a watchlist ready to paste into `conviction_leaps.ipynb`.


In [ ]:
# ══════════════════════════════════════════════════════════════════
#  🎯  STEP 7: Quick Audit Flags + Conviction Watchlist Export
# ══════════════════════════════════════════════════════════════════

display(Markdown("# 🎯 Screener Output — Quick Audit & Watchlist"))

# ── Quick audit: IV Rank + Theta estimate + Spread + Earnings flag for portfolio tickers ──
port_tickers = (
    portfolio_df["Ticker"].unique().tolist() if not portfolio_df.empty else []
)
print(f"Auditing {len(port_tickers)} portfolio tickers: {', '.join(port_tickers)}\n")

watchlist_rows = []

for ticker in port_tickers:
    try:
        t = yf.Ticker(ticker)
        time.sleep(RATE_LIMIT_SLEEP)
        hist = t.history(period="1y")
        flags = []

        # ── IV Rank (HV proxy) ──
        iv_rank = np.nan
        if not hist.empty and len(hist) >= 60:
            closes = hist["Close"].dropna()
            log_ret = np.log(closes / closes.shift(1)).dropna()
            rolling_hv = (log_ret.rolling(30).std() * math.sqrt(252)).dropna()
            if len(rolling_hv) >= 30:
                current_hv = float(rolling_hv.iloc[-1])
                hv_range = float(rolling_hv.max()) - float(rolling_hv.min())
                iv_rank = (
                    ((current_hv - float(rolling_hv.min())) / hv_range * 100)
                    if hv_range > 0
                    else 50
                )
                if iv_rank > 70:
                    flags.append("🔴 IV HIGH")
                elif iv_rank < 30:
                    flags.append("🟢 IV LOW")

        # ── Theta estimate (from best LEAPS in portfolio) ──
        pos = portfolio_df[portfolio_df["Ticker"] == ticker].iloc[0]
        spot = spot_map.get(ticker, 0)
        K = pos.get("Strike", spot)
        dte = pos.get("DTE", 365)
        mid = pos.get("Mid", 0)
        sigma = pos.get("IV", 0.30)
        if np.isnan(sigma) or sigma <= 0:
            sigma = 0.30
        T = dte / 365.0
        theta_est = (
            bsm_theta(spot, K, T, RISK_FREE_RATE, sigma)
            if spot > 0 and T > 0
            else np.nan
        )
        theta_daily_est = (
            theta_est * pos.get("Contracts", 1) * 100
            if not np.isnan(theta_est)
            else np.nan
        )
        cost = pos.get("Total Cost", 0)
        theta_pct = (
            abs(theta_daily_est) / cost * 100
            if cost > 0 and not np.isnan(theta_daily_est)
            else np.nan
        )
        if not np.isnan(theta_pct) and theta_pct > 0.3:
            flags.append(f"⚠️ Θ {theta_pct:.2f}%/day")

        # ── Spread check ──
        spread_pct = np.nan
        pos_scored = scored_df[(scored_df["Ticker"] == ticker)]
        if not pos_scored.empty:
            spread_pct = pos_scored["Spread %"].median()
            if not np.isnan(spread_pct) and spread_pct > 0.10:
                flags.append(f"⚠️ Spread {spread_pct:.0%}")

        # ── Earnings check ──
        days_to_earnings = np.nan
        try:
            cal = t.calendar
            if isinstance(cal, dict) and "Earnings Date" in cal:
                ed = cal["Earnings Date"]
                ed_list = ed if isinstance(ed, list) else [ed]
                future = [
                    (d.date() if isinstance(d, datetime) else d)
                    for d in ed_list
                    if isinstance(d, (date, datetime))
                    and (d.date() if isinstance(d, datetime) else d) >= today
                ]
                if future:
                    days_to_earnings = (sorted(future)[0] - today).days
                    if days_to_earnings < 30:
                        flags.append(f"📅 Earnings in {days_to_earnings}d")
        except Exception:
            pass

        signal = " | ".join(flags) if flags else "✅ CLEAR"

        watchlist_rows.append(
            {
                "Ticker": ticker,
                "Sector": sector_map.get(ticker, "—"),
                "Spot": spot,
                "Best LEAPS Strike": K,
                "DTE": dte,
                "IV Rank": iv_rank,
                "Θ%/day": theta_pct,
                "Spread %": spread_pct,
                "Days to Earnings": days_to_earnings,
                "Flags": signal,
            }
        )
        print(
            f"  {ticker:5s}: IV Rank={iv_rank:5.0f}  Θ%={theta_pct:.2f}%/day  → {signal}"
        )

    except Exception as e:
        print(f"  ⚠ {ticker}: {e}")
        watchlist_rows.append(
            {
                "Ticker": ticker,
                "Sector": sector_map.get(ticker, "—"),
                "Spot": spot_map.get(ticker, 0),
                "Best LEAPS Strike": np.nan,
                "DTE": np.nan,
                "IV Rank": np.nan,
                "Θ%/day": np.nan,
                "Spread %": np.nan,
                "Days to Earnings": np.nan,
                "Flags": "⚠️ Error fetching data",
            }
        )

watchlist_df = pd.DataFrame(watchlist_rows)

if not watchlist_df.empty:
    display(Markdown("### 📋 Quick Audit — Portfolio Picks"))
    display(
        watchlist_df.style.format(
            {
                "Spot": "${:,.2f}",
                "Best LEAPS Strike": "${:,.0f}",
                "IV Rank": "{:.0f}",
                "Θ%/day": "{:.2f}%",
                "Spread %": "{:.1%}",
                "Days to Earnings": "{:.0f}",
            },
            na_rep="—",
        )
        .background_gradient(subset=["IV Rank"], cmap="RdYlGn_r", vmin=0, vmax=100)
        .set_caption("Quick audit flags for screener output")
    )


# ── Watchlist export for conviction book ──
display(Markdown("---\n### 📤 Watchlist for Conviction Book"))

clean_tickers = [t for t in port_tickers if t in spot_map and spot_map[t] > 0]
display(
    Markdown(f"""
Copy this into the conviction book's input cell:

```python
CONVICTION_TICKERS = {clean_tickers}
```

Then run all cells in `conviction_leaps.ipynb` for the full deep-dive:
- Fundamentals + risk ratios
- IV smile & term structure
- IV-enhanced scoring + portfolio
- Greeks + theta decay projection
- IV rank + slippage analysis
- Earnings + exit rules + stress test + pre-trade checklist
""")
)

display(
    Markdown(f"""
### 🏁 Screener Complete — {today}

| Metric | Value |
|--------|-------|
| Universe screened | {len(UNIVERSE)} tickers |
| Passed fundamentals | {len(survivors)} |
| Options scored | {len(scored_df):,} |
| Portfolio positions | {len(portfolio_df)} |
| Capital deployed | ${deployed:,.0f} / ${STARTING_BALANCE:,} |
| **Watchlist** | **{", ".join(clean_tickers)}** |

> ⚠️ **Not financial advice.** Run the conviction book for full trading readiness before executing any trades.
""")
)